# Linear regression
Using PyTorch to perform linear regression.
An official introduction to the PyTorch library can be found here: https://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html

In [ ]:
import torch
import numpy as np
from matplotlib import pyplot as plt

## Linear regression - Manual implementation in PyTorch
In the below example we see a 1D linear regression example: $y=wx+b$

In [ ]:
x = torch.tensor(range(-5,5)).float()
x

In [ ]:
w_gt = 3
b_gt = 4
y = w_gt*x + b_gt
y

In [ ]:
w = torch.randn(1, requires_grad=True)
b = torch.randn(1, requires_grad=True)
print(f'w: {w}')
print(f'b: {b}')

Try out with different learning rate.
What happens if learning rate $0.1$ is used?

In [ ]:
learning_rate = 0.01

In [ ]:
def trainManual(w, b, lr, iternum):
    for i in range(iternum):
        y_hat = w*x + b

        error = torch.sum(torch.pow(y-y_hat,2)/y.numel())
        error.backward()   # Compute the Gradients for w and b (requires_grad=True)
        print(w,w.grad)
        # Update parameters
        with torch.no_grad():   # Temporarily set all requires_grad=False
            w -= lr * w.grad
            b -= lr * b.grad
            # Remember to zero the gradients!
            # If not, the gradients will be accumulated
            w.grad.zero_()
            b.grad.zero_()
        print("Error: {:.4f}".format(error))

In [ ]:
trainManual(w, b, learning_rate, 10)

In [ ]:
y_pred = (w*x + b)
y_pred = y_pred.data.numpy()
print("----- ----- ----- ----- -----")
print("Prediction:")
print("w_pred = {:.2f}, b_pred = {:.2f}".format(w[0] ,b[0]))
print("Ground-truth:")
print("w_gt = {:.2f}, b_gt = {:.2f}".format(w_gt ,b_gt))

In [ ]:
plt.clf()
plt.plot(x, y, 'go', label='True data', alpha=0.5)
plt.plot(x, y_pred, '--', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.show()

## Linear regression - Using the torch.nn.Module
In the below example we see a 1D linear regression example: $y=wx+b$

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn

In [ ]:
class LinearRegression(torch.nn.Module): 
    def __init__(self):
        super(LinearRegression, self).__init__() 
        self.linear = torch.nn.Linear(1, 1, bias = True) # bias is default True

    def forward(self, x):
        y_pred = self.linear(x)
        return y_pred

In [ ]:
our_model = LinearRegression()

In [ ]:
criterion = torch.nn.MSELoss()
optimizer = torch.optim.SGD(our_model.parameters(), lr = 0.01) 

In [ ]:
def trainBuildIn(model, x, y, iter):
    for i in range(iter):
        # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
        optimizer.zero_grad()
        
        # get output from the model, given the inputs
        y_pred = model(x)

        # get loss for the predicted output
        loss = criterion(y_pred, y)
        print(loss)
        # get gradients w.r.t to parameters
        loss.backward()

        # update parameters
        optimizer.step()

        print('Iter {}, loss {}'.format(iter, loss.item()))

In [ ]:
x_train = x[:,None]
y_train = y[:,None]

print(x.shape, x_train.shape)
print(y.shape, y_train.shape)

In [ ]:
trainBuildIn(our_model, x_train, y_train, 10)

In [ ]:
y_pred_bi = our_model(x_train).data.numpy()

print("----- ----- ----- ----- -----")
print("Prediction:")
for name, param in our_model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
print("Ground-truth:")
print("w_gt = {:.2f}, b_gt = {:.2f}".format(w_gt ,b_gt))

In [ ]:
plt.clf()
plt.plot(x, y, 'go', label='True data', alpha=0.5)
plt.plot(x, y_pred_bi, '--', label='Predictions', alpha=0.5)
plt.legend(loc='best')
plt.show()

# Validation with SKLEARN
Using "Ordinary least squares Linear Regression".


In [ ]:
from sklearn.linear_model import LinearRegression
linr = LinearRegression()
linr.fit(x[:,None], y[:,None])
print("----- ----- ----- ----- -----")
print("Prediction:")
print("w_pred = {:.2f}, b_pred = {:.2f}".format(float(linr.intercept_), float(linr.coef_[0])))
print("Ground-truth:")
print("w_gt = {:.2f}, b_gt = {:.2f}".format(w_gt ,b_gt))